### 외식 산업 예비 창업자를 위한 데이터 시각화 및 분석 프로그래밍

#### 데이터 불러오기(빅카인즈)

In [1]:
import pandas as pd
import json
import re
import xmltodict
import math
import os

from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException

In [2]:
class Crawling_bigkinds:
    
    click_time = 1.5
    send_time = 0.5
    date_time = 2
    wait = 10
    
    def __init__(self):
        
        prefs = {'profile.default_content_setting_values': {'cookies' : 2,
                         'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2,
                         'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2,
                         'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2,
                         'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2,
                         'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2,
                         'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}
        options = Options()
        # options.add_argument('--headless') # 출력 안되게끔 하지만 오류남 click이 위치를 못찾는것 같음
        
        self.crawling_news = dict({"Date":[],"Title":[],"Content":[]})
        
        try:
            self.driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
            sleep(self.date_time)
        except:
            sleep(self.wait)  
            self.driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
            sleep(self.date_time)
            
        self.driver.implicitly_wait(self.wait) # 완료되는 즉시 반환
        self.driver.maximize_window() # 전체화면으로 클릭하기위해

        url = "https://www.bigkinds.or.kr/"
        try:
            self.driver.get(url)
            sleep(self.date_time)
        except:
            sleep(self.wait)
            self.driver.get(url)
            sleep(self.date_time)
        
    def find_keyword(self, keyword, start, end):
        self.keyword = keyword
        self.start = start
        self.end = end
        
        # 키워드
        try:
            self.driver.find_element(By.CLASS_NAME, 'search-key').send_keys(keyword) #태그의 클래스명으로 추출
            sleep(self.send_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.CLASS_NAME, 'search-key').send_keys(keyword)
            sleep(self.send_time)
            
        # 상세분류 클릭
        try:
            self.driver.find_element(By.CLASS_NAME, 'btn-srchDetail.btn-srchDetail-search').click() # click을 사용 ->버튼클릭
            sleep(self.click_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.CLASS_NAME, 'btn-srchDetail.btn-srchDetail-search').click()
            sleep(self.click_time)
            
        # 통합분류 클릭
        try:
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.tab3.search-tab_group').click() # 공백은 .으로 구분
            sleep(self.click_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.tab3.search-tab_group').click()
            sleep(self.click_time)
        
        # 통합분류 선택
        try:
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/div/span[2]").click()
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/ul/li[2]/div/span[4]").click() # 문화 일반
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/ul/li[8]/div/span[4]").click() # 요리
            sleep(self.click_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/div/span[2]").click()
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/ul/li[2]/div/span[4]").click()
            self.driver.find_element(By.XPATH, f"//*[@id=\"srch-tab3\"]/ul/li[4]/ul/li[8]/div/span[4]").click()
            sleep(self.click_time)
            
        # 기간
        try:
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.search-tab_group').click() # 공백은 .으로 구분
            sleep(self.click_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.search-tab_group').click()
            sleep(self.click_time)
            
        # 시작 기간
        try:
            self.driver.find_element(By.ID, 'search-begin-date').send_keys(f"\b\b\b\b\b\b\b\b\b\b{start}")
            sleep(self.send_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.ID, 'search-begin-date').send_keys(f"\b\b\b\b\b\b\b\b\b\b{start}")
            sleep(self.send_time)
            
        # 끝 기간
        try:
            self.driver.find_element(By.ID, 'search-end-date').send_keys(f"\b\b\b\b\b\b\b\b\b\b{end}")
            sleep(self.send_time)
        except:
            self.driver.implicitly_wait(self.wait)
            self.driver.find_element(By.ID, 'search-end-date').send_keys(f"\b\b\b\b\b\b\b\b\b\b{end}")
            sleep(self.send_time)
        
        self.driver.find_element(By.CLASS_NAME, 'btn-srch.input-group-btn.news-search-btn').click()
        sleep(self.click_time)
            
        try:
            self.total_data = int(re.sub(',','',self.driver.find_element(By.CLASS_NAME, 'total-news-cnt').text))
        except:
            self.driver.implicitly_wait(self.wait)
            self.total_data = int(re.sub(',','',self.driver.find_element(By.CLASS_NAME, 'total-news-cnt').text))
        self.total_page = math.ceil(self.total_data/10)
        print("총 데이터 개수 : ",self.total_data)
        print("총 페이지 개수 : " ,self.total_page)
        
        
    def all_page(self):
        # 전체 페이지 크롤링
        lastpage = self.driver.find_element(By.CLASS_NAME, 'lastNum').text
        # 현재 페이지
        islastpage = False
        curpage=1
        while islastpage == False:

            ## 페이지 크롤링
            self.page_crawling()
            
            if curpage == int(lastpage):
                islastpage = True
            else:
                sleep(self.click_time)
                self.driver.find_element(By.CLASS_NAME, 'page-next.page-link').send_keys(Keys.ENTER) # click 왜 안되는가?
                curpage+=1
                sleep(self.click_time)
                
    def page_crawling(self):
        # 현재 페이지 크롤링
        
        for item in self.driver.find_elements(By.CLASS_NAME, 'thumb.news-detail'): #10개 받아서 하나씩 열기 , 섬네일로 해야 읽음 그냥 전체범위로 하면 공백때문에 에러 & click은 이미지와 매칭잘됨
            # 뉴스 클릭
            try:
                item.click()
                sleep(self.click_time)
            except:
                self.driver.implicitly_wait(self.wait)
                item.click()
                sleep(self.click_time)
                   
            #날짜
            try:
                self.driver.implicitly_wait(self.click_time)
                date = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]').text
            except:
                self.driver.implicitly_wait(self.click_time)
                date = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]').text
                sleep(self.click_time)

            #제목          
            try:
                self.driver.implicitly_wait(self.click_time)
                title = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/h1').text
            except:
                self.driver.implicitly_wait(self.click_time)
                title = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/h1').text
                sleep(self.click_time)

            #기사 내용               
            try:
                self.driver.implicitly_wait(self.click_time)
                content = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]').text
            except:
                self.driver.implicitly_wait(self.wait)
                content = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]').text
                sleep(self.click_time)
            
            # 크롤링 내용 추가
            self.crawling_news["Date"].append(date)
            self.crawling_news["Title"].append(title)
            self.crawling_news["Content"].append(content)

            #닫힘 버튼 클릭
            try:
                self.driver.find_element(By.XPATH, '//*[@id="news-detail-modal"]/div/div/button').send_keys(Keys.ENTER) #click시 에러
                sleep(self.click_time)
            except:
                self.driver.implicitly_wait(self.wait)
                self.driver.find_element(By.XPATH, '//*[@id="news-detail-modal"]/div/div/button').send_keys(Keys.ENTER)
                sleep(self.click_time)
        
    def display(self):

        print(f"키워드 : {self.keyword}, 기간 : {self.start} ~ {self.end}")
        print(f"총 페이지 수 : {self.total_page}, 총 데이터 수 : {self.total_data}")
    
    def save(self):

        start_d = self.start[:4]+self.start[5:7]+self.start[-2:]
        end_d  = self.end[:4]+self.end[5:7]+self.end[-2:]
        pd.DataFrame(self.crawling_news).to_csv(f"./cp1/NewsResult_{start_d}_{end_d}.csv", index=False, encoding="utf-8-sig") # 깨짐방지
        

In [ ]:
keyword = "외식, 음식점, 배달, 햄버거, 치킨, 피자, 마라탕, 짜장면"
start = "2022-01-04"
end = "2022-02-06"# 미만

if not os.path.exists("./cp1"):
    os.makedirs("./cp1")

Search = Crawling_bigkinds()
Search.find_keyword(keyword,start,end)
try:
    Search.all_page()
except:
    Search.driver.find_element(By.CLASS_NAME, 'page-next.page-link').send_keys(Keys.ENTER)
    Search.driver.implicitly_wait(Search.wait)
    Search.all_page()

Search.save() # 파일 저장
Search.display() # 개수 출력
Search.driver.close() # 웹페이지 닫기
sleep(Search.date_time)


In [4]:
# 크롤링 결과 확인
df = pd.read_excel('../cp1/NewsResult_20220104-20230106.xlsx', engine='openpyxl')
df

,Date,Title,Content
0,20230106,"더본코리아, 설맞이 ‘본가 소갈비찜’ RMR 제품 선봬",[아시아경제 문혜원 기자] 더본코리아는 설 명절을 맞아 온 가족이 즐길 수 있는 ‘...
1,20230106,30년 만에 최대치...손님은 지갑 닫고 업주는 '울상',[앵커]\n지난해 외식물가 상승률이 30년 만에 최대치를 기록하면서 손님은 물론이고...
2,20230106,"“레시피 없는 사찰음식, 집착 내려놓는 수행이죠”",“당신이 먹는 음식이 곧 당신이다.” \n \n 독일 철학자 루트비히 포이어바흐가 ...
3,20230106,임실엔 치~즈 맛있게 쭈~욱,전북에 가면 옛 삼국시대부터 불렸던 ‘그리운 임이 사는 곳 임실(任實)’이 있다. ...
4,20230106,음식으로 건강과 희망을 선물하는 이경애 외식연구소 대표,이경애 외식연구소 대표. 김형수기자 \n 50년 가까이 앞치마를 벗을 새 없이 앞만...
...,...,...,...
4011,20220104,[여행단신] BTS 지민이 찾은 '누웨마루 거리' 명소 등극 外,"━ 필리핀, 여행업계 오스카상 '월드 트래블 어워드' 3년 연속 선정 \n 필리핀이..."
4012,20220104,"[재택플러스] ""팁 5천 원 주세요"" 온라인 반응은?",◀ 앵커 ▶\n\n외국여행 가면 처음에 접하고 생소해하는 식당 문화가 있죠.\n\n...
4013,20220104,'맛도 정도 곱빼기' 짜장면 배달왔습니다,집안에는 40여 개의 표창장이 산처럼 쌓여 있다. 표창장의 주인공은 창원에서 '박영...
4014,20220104,장례식장서 피자 주문했는데 무슨 사연 '돈쭐 내주자',장례식장에서 피자를 주문했는데 음식점 사장이 음식 뿐 아니라 조의금과 애도를 표하는...
